In [20]:
import pandas as pd
import seaborn as sns
import scipy as stats
import requests, json
from sklearn.cross_validation import train_test_split
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import collections
import time
import csv

#make it so that we only show first 4 decimals for floats
np.set_printoptions(precision=4,suppress=True)

# visualization
%matplotlib inline
import seaborn as sns

In [19]:
# Global Variables
MIN_YEAR = 1976
MAX_YEAR = 2016

In [18]:
# Test Variables
TEST_YEAR = 2003

In [12]:
ADVANCED_PARAMS = {
    'PlayerExperience' : 'Rookie',
    'SeasonType' : 'Regular',
    'Season' : -1
    }

# cycle through all years
for year in list(range(MIN_YEAR, MAX_YEAR)):
    year = str(year) + '-' + str(year + 1)[2:4]

In [14]:
base_url = 'http://stats.nba.com/teams/advanced/#!?sort=W&dir=-1'

In [16]:
params = ADVANCED_PARAMS
params['Season'] = '2003-04'

r = requests.get(base_url, params = params)

In [22]:
driver = webdriver.Chrome()
driver.get(base_url)

WebDriverException: Message: 'chromedriver' executable needs to be in PATH. Please see https://sites.google.com/a/chromium.org/chromedriver/home
